In [5]:
import sys
!pip install opentrons
!pip install zeroconf

     |████████████████████████████████| 59 kB 3.3 MB/s eta 0:00:011
  Using cached ifaddr-0.1.7-py2.py3-none-any.whl (10 kB)


In [3]:
%load_ext autoreload
%autoreload 2

# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np
import opentrons
from  opentrons.types import Location, Point

from opentrons_helper import * 

from opentrons import simulate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Debugging


In [6]:
'''
Connect to the microscope 
'''

from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="21.3.2.3", port=5000, is_simulate=True)


Microscope is in simulation mode


In [7]:
# Connect to the opentrons robot
#protocol = opentrons.execute.get_protocol_api('2.10')
protocol = simulate.get_protocol_api('2.10')
protocol.home()

# setup the pipets
pipette = protocol.load_instrument('p300_single_gen2', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left') # causes errors -> don't use two pipettes!!

# setup the labware in the trays
plate_microscope = protocol.load_labware('corning_96_wellplate_360ul_flat', location='3')
#plate_reagents = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='9')
plate_reagents = protocol.load_labware('opentrons_15_tuberack_falcon_15ml_conical', location='10')


# Move light to the sample 
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul',location='11')

# sample plate on microscope
# offset of the well plate in mm


/Users/bene/.opentrons/robot_settings.json not found. Loading defaults
/Users/bene/.opentrons/deck_calibration.json not found. Loading defaults


In [8]:
from opentrons.types import Location, Point

# set speed
#protocol.max_speeds['X'] = 200 # default = 400
#protocol.max_speeds['Y'] = 200 # default = 300

# ######
# 0 4 8
# 1 5 9
# 2 6 10 
# 3 7 11

# wells with reagents
i_yeast = 0
i_peptone = 1
i_sugar = 2
i_water = 3


# define timig
t_bsa = 15
t_primary = 60
t_secondary = 60
t_fluo = 30*0

# move it down since opentrons expects the reagents to be furthter away from the ground
offset_z_reagents = -65

# define locations 
minimum_z_height = 150 # minimum height for arcs driven by the robot

# compute the positions
position_trash =  (365,350, 100)

position_idle =  (100,100, 150)#

# Calibrate

In [10]:
'''
calibrate tiprack
'''
#protocol.home()
i_pipette = 1
#pitty, but moving the xy coordinates doesn't work
print(i_pipette)
offset_pipette_rack=(1,3,0)
tiprack.set_calibration(Point(*offset_pipette_rack))

i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)
print(type(i_pipette))

1
My pipette number: 1and type: <class 'int'>
<class 'int'>


In [11]:
#from opentrons_client import OpentronsClient 
#ot2 = OpentronsClient(host="21.3.2.4", port=31950)
#ot2.toggle_light(True)
#ipot2.pipette_home(location="right")
#pos = ot2.positions()

In [12]:
'''
calibrate illumination poisiton
'''
microscope.home()
#protocol.home()
#position_idle =  (100,300, 150)#
#move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
well_to_well_steps= 9100
#microscope.home()
position_sample_light = (250, 99, 190)

# This offset represents the position of well 7!
offset_x = 4000 # 1290+0*9000
offset_y = -1000# -1400+4*9000
offset_z = microscope.position["z"]
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
microscope.set_laser_led(0,1)
microscope.move((offset_x,offset_y,offset_z),absolute=True)
autofocus_dz = 500
autofocus_Nz = 7
is_autofocus = False
#offset_z = microscope.autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
#microscope.set_laser_led(0,1)


microscope.move((offset_x,offset_y,offset_z),absolute=True)

Microscope is homing...
Please add a pipette first!
Moving to: (250, 99, 190)


In [13]:

microscope.set_laser_led(0,1)

In [14]:
'''
Calibrate Wellposition for sample coordinates (96well plate)
pipette should dip in the wells on the microscope

'''
#pipette.home()
# lets assume the robots light source is in place 
microscope.set_laser_led(i_laser=0, i_led=1)


# lets define the position for the seventh well (ID)
offset_x = 5000
offset_y = -1000
offset_z = 1500

microscope.move((offset_x, offset_y, offset_z))

print(microscope.position)

offset_z
I_laser = 0
I_led = 1
id = 0

offset_z = microscope.position["z"]
home_position_microscope=(0,0,offset_z)
microscope.move((home_position_microscope))

minimum_z_height = 155
offset_plate_sample=(-28,68,127)
plate_microscope.set_calibration(Point(*offset_plate_sample))

for i_pos in range(4):
        position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
        print(position_sample_i)
        move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)




[251.38  142.24  135.885]
Moving to: (251.38, 142.24, 135.885)
[251.38  133.24  135.885]
Moving to: (251.38, 133.24, 135.885)
[251.38  124.24  135.885]
Moving to: (251.38, 124.24000000000001, 135.885)
[251.38  115.24  135.885]
Moving to: (251.38, 115.24000000000001, 135.885)
Moving to: (251.38, 115.24000000000001, 145.885)


In [15]:
# goes into all 3 reagent slots 
'''
calibrate reagetn slots
'''
pipette.home()

offset_x_reagents = -4
offset_y_reagents = 8
offset_z_reagents = -50 # smaller number means: go up
for i_pos in range(3):
    #i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)
    position_reagent_i = np.array(plate_reagents.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)

# move out of the well again
move_to_coord(pipette, position_reagent_i, offset=(0,0,20), minimum_z_height=minimum_z_height)


Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (14.21, 335.65, 10.949999999999996)
Moving to: (14.21, 316.37, 10.949999999999996)
Moving to: (18.21, 308.37, 80.94999999999999)


# Start the experiment

In [16]:
# initialize strings
from datetime import datetime
timestamp = datetime.now().strftime("%Y_%m_%d-%I-%M-%S_%p")
i_experiment = 0



'''
##############################################################################
##########################HOW MANY WELLS DO WE WANT TO DISPENSE?!##########################
##############################################################################
'''



In [26]:
'''
Set the constant values for the ingredients in the different bottles
'''


V_aspirate = 200e-6 # volume per dispensing step => we have 3 steps and want to always have 600µl in the end 
V_flask = 15e-3 # Volume inside one falcon tube 
c_peptone = 10 # g/L
c_yeast = 10 # g/L
c_sach = 100 # g/L
c_sach_var = (0,1,2,5,10,20,50,100) # Concentration we would like to vary in different wells g/L
N_replicates = 3 # how many times we would like to replicate each concentration combination per experiment to avoid bad data?

well_id_list = np.arange(0,len(c_sach_var)*N_replicates) # we don't want to go to the very end.



# compute the solution in the 15ml bottle:
# ammount in flask in gramm
a_yeast = c_yeast*V_flask
a_peptone = c_peptone*V_flask
a_sach = c_sach*V_flask



print("Prepare yeast: ",c_yeast*V_flask, "g per 15ml bottle")
print("Prepare peptone: ",c_peptone*V_flask, "g per 15ml bottle")
print("Prepare sachharose: ",c_sach*V_flask, "g per 15ml bottle")




Prepare yeast:  0.15 g per 15ml bottle
Prepare peptone:  0.15 g per 15ml bottle
Prepare sachharose:  1.5 g per 15ml bottle


In [27]:
'''
Protocol in a nutshell
Quick simulation if everything works as it should
'''


'''
final concentration for sachharose if we mix it with water:
    C=(a1+a2)/(V1+V2)
     =>
     C_fina l = (C_stock*V_sachh)/(V_sachh+V_water)
     V_aspirate = V_sachh+V_water
     =>
     V_water = V_aspirate*(C_final-C_stock)/C_stock
'''
#%%
print("Stock solution: (Yeast, Peptone, Sachharose:")
print("Yeast, | ,  Peptone,  | , Sachharose:")
print("[g/L]"," | ", "[g/L]"," | ", "[g/L]")

for c_i_sach in c_sach_var:
    #compute amount of water to end up with concentration for sachharose
    V_water = V_aspirate*(c_sach-c_i_sach)/c_sach
    V_sachh = V_aspirate-V_water
    print("V_water",round(V_water*1e6,1))
    print("V_sachh",round(V_sachh*1e6,1))

#    print("V_yeast : ", V_aspirate*1e6, "V_peptone: ", V_aspirate*1e6, "V_sachh: ",V_sachh*1e6, "V_water: ", V_water*1e6, "µl")   
    print((V_aspirate*c_yeast)/V_aspirate," | ", V_aspirate*c_peptone/V_aspirate, " | ",  round((c_sach*V_sachh)/V_aspirate,2))


Stock solution: (Yeast, Peptone, Sachharose:
Yeast, | ,  Peptone,  | , Sachharose:
[g/L]  |  [g/L]  |  [g/L]
V_water 200.0
V_sachh 0.0
10.0  |  10.0  |  0.0
V_water 198.0
V_sachh 2.0
10.0  |  10.0  |  1.0
V_water 196.0
V_sachh 4.0
10.0  |  10.0  |  2.0
V_water 190.0
V_sachh 10.0
10.0  |  10.0  |  5.0
V_water 180.0
V_sachh 20.0
10.0  |  10.0  |  10.0
V_water 160.0
V_sachh 40.0
10.0  |  10.0  |  20.0
V_water 100.0
V_sachh 100.0
10.0  |  10.0  |  50.0
V_water 0.0
V_sachh 200.0
10.0  |  10.0  |  100.0


## Add Yeast solution ( 0.15g/15mL) -> 200 uL per pipette to the sample plate

PLEASE ADD SOME COMMENTS HERE

First we want to get the yeast spread to all the wells!

In [42]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

expt_no = 0

for i_pos in range(len(well_id_list)):
    # get yeast from the flask
    print("Pickup Yeast")
    position_reagent_i = np.array(plate_reagents.wells()[i_yeast].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # add yeast to sample
    print("Deliver Yeast")
    position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)
    time.sleep(1)

move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)



My pipette number: 8and type: <class 'int'>
Pickup Yeast
Moving to: (14.21, 354.93, 10.949999999999996)
Deliver Yeast
Moving to: (251.38, 142.24, 135.885)
Moving to: (251.38, 142.24, 145.885)
Pickup Yeast
Moving to: (14.21, 354.93, 10.949999999999996)
Deliver Yeast
Moving to: (251.38, 133.24, 135.885)
Moving to: (251.38, 133.24, 145.885)
Pickup Yeast
Moving to: (14.21, 354.93, 10.949999999999996)
Deliver Yeast
Moving to: (251.38, 124.24000000000001, 135.885)
Moving to: (251.38, 124.24000000000001, 145.885)
Pickup Yeast
Moving to: (14.21, 354.93, 10.949999999999996)
Deliver Yeast
Moving to: (251.38, 115.24000000000001, 135.885)
Moving to: (251.38, 115.24000000000001, 145.885)
Pickup Yeast
Moving to: (14.21, 354.93, 10.949999999999996)
Deliver Yeast
Moving to: (251.38, 106.24000000000001, 135.885)
Moving to: (251.38, 106.24000000000001, 145.885)
Pickup Yeast
Moving to: (14.21, 354.93, 10.949999999999996)
Deliver Yeast
Moving to: (251.38, 97.24, 135.885)
Moving to: (251.38, 97.24, 145.885

##  Add peptone solution ( 0.15g/15mL) -> 200 uL per pipette to the sample plate

PERHAPS WE CAN ALSO MAKE THIS HERE "ADAPTIVE"? SO THAT YOU ENTER THE EXPERIMENT ID AND THE VALUE WILL AUTOMATICALLY BE EXPORTED/STORED? I KNOW THAT YOU WILL PREPARE THE BOTTLE WITH THE PEPTONE BEFOREHAND


ADD SOME COMMENTS HERE!

```py
V_aspirate_peptone = 200 #uL
```

Here we will spread a constant (!) ammount of petone to every well which will varry from day to day:
0,1,2,5,10,20,50,100,200 g/L

In [32]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

for i_pos in range(len(well_id_list)):
    # get peptone
    print("Pickup peptone")
    position_reagent_i = np.array(plate_reagents.wells()[i_peptone].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # add yeast to sample
    print("Spread peptone")
    position_sample_i = np.array(plate_microscope.wells()[c_peptone].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    time.sleep(1)

move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)



My pipette number: 6and type: <class 'int'>
Pickup peptone
Moving to: (14.21, 335.65, 10.949999999999996)
Spread peptone
Moving to: (251.38, 142.24, 135.885)
Pickup peptone
Moving to: (14.21, 335.65, 10.949999999999996)
Spread peptone
Moving to: (251.38, 142.24, 135.885)
Pickup peptone
Moving to: (14.21, 335.65, 10.949999999999996)
Spread peptone
Moving to: (251.38, 142.24, 135.885)
Pickup peptone
Moving to: (14.21, 335.65, 10.949999999999996)
Spread peptone
Moving to: (251.38, 142.24, 135.885)
Pickup peptone
Moving to: (14.21, 335.65, 10.949999999999996)
Spread peptone
Moving to: (251.38, 142.24, 135.885)
Pickup peptone
Moving to: (14.21, 335.65, 10.949999999999996)
Spread peptone
Moving to: (251.38, 142.24, 135.885)
Pickup peptone
Moving to: (14.21, 335.65, 10.949999999999996)
Spread peptone
Moving to: (251.38, 142.24, 135.885)
Pickup peptone
Moving to: (14.21, 335.65, 10.949999999999996)
Spread peptone
Moving to: (251.38, 142.24, 135.885)
Pickup peptone
Moving to: (14.21, 335.65, 10

7

## Add sugar solution( 1.5g/15mL) plus water -> variable uL per pipette to the sample plate

In [44]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

'''
final concentration for sachharose if we mix it with water:
    C=(a1+a2)/(V1+V2)
     =>
     C_fina l = (C_stock*V_sachh)/(V_sachh+V_water)
     V_aspirate = V_sachh+V_water
     =>
     V_water = V_aspirate*(C_final-C_stock)/C_stock
'''
#%%
print("Stock solution: (Yeast, Peptone, Sachharose:")
print("Yeast, | ,  Peptone,  | , Sachharose:")
print("[g/L]"," | ", "[g/L]"," | ", "[g/L]")
expt_no = 0

for i_water_sachh in range(2):
    # first distribute the water, then the sachharose

    # iterate over ALL wells in one plate 
    for i_pos in range(len(well_id_list)):
        # We want to replicate the exerpiment after one round, so there will be an experiment 
        # with same concentration after len(well_id_list)-times! 
        c_sach_i = c_sach_var[i_pos%len(c_sach_var)]
        
        #compute amount of water to end up with concentration for sachharose
        V_water = V_aspirate*(c_sach-c_sach_i)/c_sach
        V_sachh = V_aspirate-V_water

        if i_water_sachh == 0:
            # we go for water
            print("Aspirating water", round(V_water*1e6,1))
            V_sample = V_water
            i_reagent = i_water
        if i_water_sachh == 1:
            # we go for water
            print("Aspirating Sachharose", round(V_water*1e6,1))
            V_sample = V_sachh
            i_reagent = i_sugar
                        
        # get sugar
        position_reagent_i = np.array(plate_reagents.wells()[i_reagent].center()._point)
        move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
        pipette.aspirate(V_sample)

        # add sugar to sample 
        position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
        move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
        pipette.dispense()
        time.sleep(1)


    
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)



My pipette number: 10and type: <class 'int'>
Stock solution: (Yeast, Peptone, Sachharose:
Yeast, | ,  Peptone,  | , Sachharose:
[g/L]  |  [g/L]  |  [g/L]
Aspirating water 200.0
Moving to: (34.1, 335.65, 10.949999999999996)
Moving to: (251.38, 142.24, 135.885)
Aspirating water 198.0
Moving to: (34.1, 335.65, 10.949999999999996)
Moving to: (251.38, 133.24, 135.885)
Aspirating water 196.0
Moving to: (34.1, 335.65, 10.949999999999996)
Moving to: (251.38, 124.24000000000001, 135.885)
Aspirating water 190.0
Moving to: (34.1, 335.65, 10.949999999999996)
Moving to: (251.38, 115.24000000000001, 135.885)
Aspirating water 180.0
Moving to: (34.1, 335.65, 10.949999999999996)
Moving to: (251.38, 106.24000000000001, 135.885)
Aspirating water 160.0
Moving to: (34.1, 335.65, 10.949999999999996)
Moving to: (251.38, 97.24, 135.885)
Aspirating water 100.0
Moving to: (34.1, 335.65, 10.949999999999996)
Moving to: (251.38, 88.24, 135.885)
Aspirating water 0.0
Moving to: (34.1, 335.65, 10.949999999999996)
Mov

# Perform a timelapse image series 

In [50]:
#microscope.plate_shaking(1)
num_cell_list = []
autofocus_dz = 1000
autofocus_Nz= 7
mytime = -time.time()
#t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list

t_duration = 60 # how long does the experiment preform?
t_period = 5 # min - how often should the expimrent be carried out?
is_autofocus = True
is_find_prefocus = False# True
    
i_experiment = 0

print(i_experiment)
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)


wellposlist = well_id_list


name_experiment = timestamp+"_Cyeast_"+str(c_yeast)+"_Cpeptone_"+str(c_peptone)+"_Csachhrose_"+str(min(c_sach_var))+"-"+str(max(c_sach_var))


# perform a testing well scan by moving robot to light position and do a whole plate scan
#move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
focus_pos_list = wellscan_list(microscope, 
        offset_x, 
        offset_y, 
        i_experiment,
        well_id_list,
        well_to_well_steps,
        autofocus_dz, 
        autofocus_Nz,
        name_experiment=name_experiment,
        focus_pos_list=focus_pos_list, 
        is_find_prefocus=is_find_prefocus,
        is_autofocus=is_autofocus,
        is_autofocus_fine = False, 
        is_autofocus_fast = False,
        I_laser=0, I_led=1,
        t_debounce=0.5)



            
            
        

0
Please add a pipette first!
Moving to: (250, 99, 190)
Start scan


UnboundLocalError: local variable 'offset_z' referenced before assignment

## 12 hour time lapse

'2021_06_23-12-15-34_PM_Cyeast_10_Cpeptone_0_Csachhrose_0-100'

## Microscope position config

{'x': 0, 'y': 0, 'z': 0}


## Take focus-defocus images after time-lapse 

In [49]:
# create the object for the well scanner 
wellscanner = Hi2Module( microscope, well_to_well_steps=9000,
             N_wells = 96, Nx = 8, Ny = 12)

# set the well-position of the well number  7 
#wellscanner.set_offset_for_well_id_7(offset_x, offset_y)


microscope.home()
for id in range(0,95):

    # first go to well ID 
    wellscanner.move_to_well_id(offset_z=None, well_index=id)
    
    # get the acutal position of the microscope
    current_x, current_y, _ = microscope.position['x'],  microscope.position['y'],  microscope.position['z']
    
   # current_x += np.radnom.rnadniintcurrent_x+ 500
    
    microscope.set_laser_led(I_laser,I_led)
    
    offset_z = microscope.autofocus_coarse(dz=1000, nz=15)
    current_x, current_y, _ = microscope.position['x'],  microscope.position['y'],  microscope.position['z']
    

    # 1. capture the in-focus image
    base_filename = 'data_generation_GAN_17Jun_'
    params = {
        "filename": base_filename+"de_focus_"+str(id),
    }
    microscope.set_laser_led(I_laser,I_led)
    microscope.move((offset_x, offset_y, offset_z))
    time.sleep(1)
    microscope.capture_image_to_disk(params)

    max_dz = 200 # the maximum value the focus may differ => plus minus max_dz/2
    offset_z_defocus = offset_z+ np.random.randint(max_dz)-(max_dz//2)
    print(offset_z_defocus)

    # 2. cpature the defocus image 
    params = {
        "filename": base_filename+"in_focus_"+str(id),
    }

    microscope.move((current_x, current_y, offset_z_defocus))
    time.sleep(1) # debounce any vibvration
    microscope.capture_image_to_disk(params)



NameError: name 'Hi2Module' is not defined

### Additional steps : 

In [ ]:
# Do we need this ?

#microscope.plate_shaking(1)
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
mytime = -time.time()

t_period = 20 # min - how often should the expimrent be carried out?
is_autofocus = True
is_find_prefocus = True
autofocus_dz = 1500
autofocus_Nz = 15
#i_experiment = 0
focus_pos_list = []
while True:

    if time.time()-mytime > (t_period*60):
        mytime = time.time()
        i_experiment += 1
    
        # perform a testing well scan by moving robot to light position and do a whole plate scan
        #move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
        wellscanner.wellscan_list(
                    i_experiment,
                    well_id_list,
                    autofocus_dz, 
                    autofocus_Nz,
                    name_experiment="wellscan_yeast__new_",
                    is_autofocus=is_autofocus, 
                    I_laser=0, 
                    I_led=1,
                    t_debounce=.5,
                    is_find_prefocus=is_find_prefocus,
                    process_func=None)
        is_autofocus = False
        is_find_prefocus = False
        
    else:
        time.sleep(1)
        

    



            
        

In [ ]:
microscope.set_laser_led(0,1)
wellscanner.move_to_well_id(7)